# MergeSpectroPhotoFactory

- creation date : July 4th 2019
- update : 8 july 2019
- author Sylvie Dagoret

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

import datetime
from dateutil import parser   # parser of string into datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import matplotlib.dates as mdates
from matplotlib import gridspec

import numpy as np
import os
from scipy.interpolate import interp1d

from astropy.time import Time


In [5]:
#old = np.load
#np.load = lambda *a,**k: old(*a,**k,allow_pickle=True)

In [6]:
from astropy.table import Table,QTable

In [7]:
plt.rcParams["axes.labelsize"]="large"
plt.rcParams["axes.linewidth"]=2.0
plt.rcParams["xtick.major.size"]=8
plt.rcParams["ytick.major.size"]=8
plt.rcParams["ytick.minor.size"]=5
plt.rcParams["xtick.labelsize"]="large"
plt.rcParams["ytick.labelsize"]="large"

plt.rcParams["figure.figsize"]=(10,10)
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['axes.titleweight'] = 'bold'
#plt.rcParams['axes.facecolor'] = 'blue'
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['lines.markeredgewidth'] = 0.3 # the line width around the marker symbol
plt.rcParams['lines.markersize'] = 5  # markersize, in points
plt.rcParams['grid.alpha'] = 0.75 # transparency, between 0.0 and 1.0
plt.rcParams['grid.linestyle'] = '-' # simple line
plt.rcParams['grid.linewidth'] = 0.4 # in points
plt.rcParams['font.size'] = 13

In [8]:
import os
from os import listdir
from os.path import isfile, join
import re
import sys

if not 'workbookDir' in globals():
    workbookDir = os.getcwd()
print('workbookDir: ' + workbookDir)

spectractordir=workbookDir+"/../../Spectractor"
print('spectractordir: ' + spectractordir)
toolsdir=workbookDir+"/../common_tools"
print("toolsdir:",toolsdir)


sys.path.append(workbookDir)
sys.path.append(spectractordir)
sys.path.append(os.path.dirname(workbookDir))
sys.path.append(toolsdir)

from libatmscattering import *

workbookDir: /Users/dagoret/MacOSX/GitHub/LSST/SpectractorAnaAtm19/ana_20190215_HD116405_Filtre_None
spectractordir: /Users/dagoret/MacOSX/GitHub/LSST/SpectractorAnaAtm19/ana_20190215_HD116405_Filtre_None/../../Spectractor
toolsdir: /Users/dagoret/MacOSX/GitHub/LSST/SpectractorAnaAtm19/ana_20190215_HD116405_Filtre_None/../common_tools


In [9]:
def GetTables(tablesdir):
    """
    """
    fn_info=os.path.join(tablesdir,"Info.npy")
    fn_wl=os.path.join(tablesdir,"Lambdas_ref.npy")
    fn_msk=os.path.join(tablesdir,"Mask.npy")
    fn_att=os.path.join(tablesdir,"MAttenuation_mean_ALL.npy")
    fn_atterr=os.path.join(tablesdir,"MAttenuation_Err_ALL.npy")

    general_info=np.load(fn_info,allow_pickle=True)
    Lambdas_ref=np.load(fn_wl,allow_pickle=True)
    mask= np.load(fn_msk,allow_pickle=True)
    
    #read masked arrays
    compressed=np.load(fn_att)
    values = np.zeros_like(mask, dtype=compressed.dtype)
    np.place(values, ~mask, compressed)
    MAttenuation_mean_ALL = np.ma.MaskedArray(values, mask)
    
    compressed=np.load(fn_atterr)
    values = np.zeros_like(mask, dtype=compressed.dtype)
    np.place(values, ~mask, compressed)
    MAttenuation_Err_ALL = np.ma.MaskedArray(values, mask)
    
    
    return general_info,Lambdas_ref,MAttenuation_mean_ALL,MAttenuation_Err_ALL
   

In [10]:
def GetOtherTables(tablesdir):
    """
    """
    
    fn_msk_mag=os.path.join(tablesdir,"MMagMask.npy")
    fn_mag=os.path.join(tablesdir,"MMagnitude_mean_ALL.npy")
    fn_magerr=os.path.join(tablesdir,"MMagnitude_Err_ALL.npy")

    fn_msk_abs=os.path.join(tablesdir,"MAbsMask.npy")                 
    fn_abs=os.path.join(tablesdir,"MAbsorption_mean_ALL.npy")
    fn_abserr=os.path.join(tablesdir,"MAbsorption_Err_ALL.npy")
                
    
    mask1= np.load(fn_msk_mag,allow_pickle=True)
    mask2= np.load(fn_msk_abs,allow_pickle=True)
    
    #read masked arrays
    compressed=np.load(fn_mag)
    values = np.zeros_like(mask1, dtype=compressed.dtype)
    np.place(values, ~mask1, compressed)
    MMag_mean_ALL = np.ma.MaskedArray(values, mask1)
    
    compressed=np.load(fn_magerr)
    values = np.zeros_like(mask1, dtype=compressed.dtype)
    np.place(values, ~mask1, compressed)
    MMag_Err_ALL = np.ma.MaskedArray(values, mask1)
    
    #read masked arrays
    compressed=np.load(fn_abs)
    values = np.zeros_like(mask2, dtype=compressed.dtype)
    np.place(values, ~mask2, compressed)
    MAbs_mean_ALL = np.ma.MaskedArray(values, mask2)
    
    compressed=np.load(fn_abserr)
    values = np.zeros_like(mask2, dtype=compressed.dtype)
    np.place(values, ~mask2, compressed)
    MAbs_Err_ALL = np.ma.MaskedArray(values, mask2)
    
    
    return MMag_mean_ALL, MMag_Err_ALL, MAbs_mean_ALL,MAbs_Err_ALL

In [27]:
def ExtractFromFilename(file_str):
    """
    """
    SearchTagRe_date = '^T1M_([0-9]+)_.*_red_spectrum[.]fits$'
    SearchTagRe_time = '^T1M_[0-9]+_([0-9]+)_.*_red[.]fits$'
    SearchTagRe_num = '^T1M_[0-9]+_[0-9]+_([0-9]+)_.*_red[.]fits$'
    SearchTagRe_obj = '^T1M_[0-9]+_[0-9]+_[0-9]+_(.*)_Filtre.*_red[.]fits$'
    SearchTagRe_disp = '^T1M_[0-9]+_[0-9]+_[0-9]+_.*_(.*)_Filtre.*_red[.]fits$'
    SearchTagRe_filt = '^T1M_[0-9]+_[0-9]+_[0-9]+_.*_(Filtre.*)[.][0-9]+_red[.]fits$'
    SearchTagRe_evnum = '^T1M_[0-9]+_[0-9]+_[0-9]+_.*_Filtre.*[.]([0-9]+)_red[.]fits$'
    
    
    tag_date=re.findall(SearchTagRe_date, file_str)
    tag_time=re.findall(SearchTagRe_time, file_str)
    
    if len(tag_date)==1 and len(tag_time)==1:
        return tag_date[0],tag_time[0]
    else:
        return None,None


# Selected observations close to the hologram symetry center

## Selection file form Marc

In [11]:
selection_file="Obs_selections/liste-Dnominal-lt-100.xlsx"

In [12]:
df=pd.read_excel(selection_file,header=1)

In [13]:
df

,date,airmass,starmag,bkgmag,starmagerr,bkgmagerr,x,y,file,Dnominal
0,"2019-02-15T22:55:50,730",1.545961,-16.095720,-13.549041,0.000317,0.000968,468.679029,72.263177,T1M_20190215_234735_303_HD116405_Filtre_None_b...,12.122409
1,"2019-02-15T22:56:26,007",1.542936,-16.094987,-13.544812,0.000317,0.000970,463.709445,67.179371,T1M_20190215_234700_026_HD116405_Filtre_None_b...,12.234532
2,"2019-02-15T22:57:01,284",1.539925,-16.095924,-13.543315,0.000317,0.000970,463.597590,65.805203,T1M_20190215_234624_749_HD116405_Filtre_None_b...,13.418125
3,"2019-02-15T22:57:36,562",1.536929,-16.038728,-13.535786,0.000317,0.000974,462.710382,63.056925,T1M_20190215_234549_469_HD116405_Filtre_None_b...,15.667903
4,"2019-02-15T22:58:11,838",1.533947,-16.042788,-13.978634,0.000306,0.000794,459.274401,93.777449,T1M_20190216_001648_135_HD116405_Filtre_None_b...,15.828834
5,"2019-02-15T22:58:47,115",1.530979,-16.036012,-13.559866,0.000317,0.000963,474.807268,76.894069,T1M_20190215_234810_580_HD116405_Filtre_None_b...,16.843615
6,"2019-02-15T22:59:22,399",1.528025,-16.041902,-13.548013,0.000315,0.000968,475.434865,77.214376,T1M_20190215_234845_858_HD116405_Filtre_None_b...,17.452556
7,"2019-02-15T22:59:57,677",1.525086,-16.036774,-13.526483,0.000318,0.000978,456.562442,59.547702,T1M_20190215_234514_107_HD116405_Filtre_None_b...,18.508211
8,"2019-02-15T23:00:32,956",1.522160,-16.033645,-13.563831,0.000316,0.000961,478.171347,78.763333,T1M_20190215_234921_136_HD116405_Filtre_None_b...,20.185785
9,"2019-02-15T23:01:08,233",1.519248,-16.033850,-13.616570,0.000319,0.000938,483.721798,78.083421,T1M_20190215_234956_414_HD116405_Filtre_None_b...,25.721934


In [14]:
selected_datetime = [ parser.parse(date_str) for date_str in  df["date"].values]

In [15]:
selected_datetime=np.array(selected_datetime)

In [16]:
selected_datetime.sort()

In [17]:
selected_datetime

array([datetime.datetime(2019, 2, 15, 22, 55, 50, 730000),
       datetime.datetime(2019, 2, 15, 22, 56, 26, 7000),
       datetime.datetime(2019, 2, 15, 22, 57, 1, 284000),
       datetime.datetime(2019, 2, 15, 22, 57, 36, 562000),
       datetime.datetime(2019, 2, 15, 22, 58, 11, 838000),
       datetime.datetime(2019, 2, 15, 22, 58, 47, 115000),
       datetime.datetime(2019, 2, 15, 22, 59, 22, 399000),
       datetime.datetime(2019, 2, 15, 22, 59, 57, 677000),
       datetime.datetime(2019, 2, 15, 23, 0, 32, 956000),
       datetime.datetime(2019, 2, 15, 23, 1, 8, 233000),
       datetime.datetime(2019, 2, 15, 23, 1, 43, 511000),
       datetime.datetime(2019, 2, 15, 23, 2, 18, 787000),
       datetime.datetime(2019, 2, 15, 23, 2, 54, 99000),
       datetime.datetime(2019, 2, 15, 23, 3, 29, 374000),
       datetime.datetime(2019, 2, 15, 23, 4, 4, 652000),
       datetime.datetime(2019, 2, 15, 23, 4, 39, 930000),
       datetime.datetime(2019, 2, 15, 23, 5, 15, 206000),
       datet

In [21]:
file_str=df["file"][0]

In [22]:
file_str

'T1M_20190215_234735_303_HD116405_Filtre_None_bin1x1,89_red_spectrum,fits'

In [23]:
SearchTagRe_date = '^T1M_([0-9]+)_.*_red_spectrum[.]fits$'
SearchTagRe_time = '^T1M_[0-9]+_([0-9]+)_.*_red[.]fits$'
SearchTagRe_num = '^T1M_[0-9]+_[0-9]+_([0-9]+)_.*_red[.]fits$'
SearchTagRe_obj = '^T1M_[0-9]+_[0-9]+_[0-9]+_(.*)_Filtre.*_red[.]fits$'
SearchTagRe_disp = '^T1M_[0-9]+_[0-9]+_[0-9]+_.*_(.*)_Filtre.*_red[.]fits$'
SearchTagRe_filt = '^T1M_[0-9]+_[0-9]+_[0-9]+_.*_(Filtre.*)[.][0-9]+_red[.]fits$'
SearchTagRe_evnum = '^T1M_[0-9]+_[0-9]+_[0-9]+_.*_Filtre.*[.]([0-9]+)_red[.]fits$'

In [25]:
tag_date=re.findall(SearchTagRe_date, file_str)
tag_time=re.findall(SearchTagRe_time, file_str)

In [26]:
tag_date

[]

# Import Photometric Data

In [ ]:
t = Table.read('out_starphotometryfactory/table_starphotometry.ecsv', format='ascii.ecsv')

In [ ]:
t[:5]

In [ ]:
t["starmag"]=-2.5*np.log10(t["sigcirc"])
t["bkgmag"]=-2.5*np.log10(t["bkgsum"])

t["starmagerr"]=2.5/2.3/t["sigcirc"]*np.sqrt(t["errstatcirc"]**2+t["errannul"]**2)
t["bkgmagerr"]=2.5/2.3/t["bkgsum"]*t["bkgstddevpix"]*2000.

In [ ]:
Nobs1=len(t)

In [ ]:
Nobs1

## Convert in datetime

- necessary for plotting

In [ ]:
all_datetime1 = [Time(d,format='isot', scale='utc').to_datetime()  for d in t["date"]]

# Import spectrometric data

In [ ]:
tabledir2="out_processgreyattenuation"

In [ ]:
general_info,Lambdas_ref,MAttenuation_mean_ALL,MAttenuation_Err_ALL=GetTables(tabledir2)

In [ ]:
MAttenuation_mean_ALL

In [ ]:
all_indexes2=general_info[:,0]
all_eventnum2=general_info[:,1]
all_airmass2=general_info[:,2]
all_dt2=general_info[:,3]
all_datetime2=general_info[:,4]
all_referencebasedattenuation2=general_info[:,5]
all_flag2=general_info[:,6]

In [ ]:
NWLBIN,Nobs2=MAttenuation_mean_ALL.shape

## Reference point

In [ ]:
t2=Table.read(os.path.join(tabledir2,"ReferencePoint.ecsv"), format='ascii.ecsv')

In [ ]:
t2

In [ ]:
idx_ref2=t2["idx"][0]

In [ ]:
datetime_ref=Time(t2["date"][0]).to_datetime()

In [ ]:
datetime_ref

In [ ]:
all_datetime1=np.array(all_datetime1)

In [ ]:
idx_ref=np.where(all_datetime1==datetime_ref)[0][0]

In [ ]:
t[idx_ref]

# Association

In [ ]:
map_photo_to_spectro_idx= np.array([ np.where(all_datetime2==time1)[0][0] for time1 in all_datetime1])

In [ ]:
map_photo_to_spectro_idx

# Try correlations

## Select the wavelength

In [ ]:
WL_Selected=np.array([400,500,600,700,800,900])
NBSELW=len(WL_Selected)

In [ ]:
Lambdas_ref

In [ ]:
#idx = (np.abs(A-value)).argmin()
# X = np.abs(A-value) ; idx = np.where( X == X.min() )
all_idx = [ (np.abs(Lambdas_ref-WL_Selected[iw])).argmin() for iw in np.arange(NBSELW) ]

In [ ]:
all_idx=np.array(all_idx)

In [ ]:
Lambdas_ref[all_idx]

In [ ]:
# wavelength bin colors
jet = plt.get_cmap('jet')
cNorm = colors.Normalize(vmin=0, vmax=NBSELW)
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=jet)
all_colors = scalarMap.to_rgba(np.arange(NBSELW), alpha=1)

In [ ]:
plt.figure(figsize=(20,12))

gs = gridspec.GridSpec(2, 1,height_ratios=[1, 2])

ax1 = plt.subplot(gs[0,0])
ax1.plot(all_datetime1,t["starmag"]-t["starmag"][idx_ref],'ko',label="star photo")
#ax1.set_ylim(-16.5,-15.5)
ax1.set_ylim(-0.5,0.5)
ax1.plot([all_datetime1[idx_ref],all_datetime1[idx_ref]],[-0.5,0.5],'g:')


ax1.set_ylabel("photo mag")
ax1.grid()
ax1.legend(fontsize=15)
ax2 = plt.subplot(gs[1,0],sharex=ax1)

count=0
for iwl in all_idx:

    ax2.plot(all_datetime1,MAttenuation_mean_ALL[iwl,map_photo_to_spectro_idx],'o',
             color=all_colors[count],label="{:3.0f} nm".format(Lambdas_ref[iwl]))
    
    count+=1
ax2.plot([all_datetime1[idx_ref],all_datetime1[idx_ref]],[-1,1],'g:')


ax2.grid()
ax2.legend(fontsize=15)
ax2.set_ylabel("spectro mag")
ax2.set_ylim(-1.,1.)
ax2.set_xlabel("date")
plt.suptitle("Comparison of magnitudes : Star Photometry - Spectroscopy",fontweight="bold",fontsize=20)

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(all_datetime1,t["starmag"]-t["starmag"][idx_ref],'bo',label="star photometry")
plt.plot(all_datetime1,MAttenuation_mean_ALL[all_idx[0],map_photo_to_spectro_idx],'ro',label="spectrometry $\lambda$={:3.0f} nm".format(Lambdas_ref[all_idx[0]]))
plt.ylim(-0.75,0.75)
plt.grid()
plt.legend()
colors=["b","g","r"]
ii=0
for wl in [400.,500.,600.]:
    od_adiab = RayOptDepth_adiabatic(wl,altitude=2890.5, costh=1./t["airmass"]) - RayOptDepth_adiabatic(wl,altitude=2890.5, costh=1.) # Rayleigh optical depth
    label="Rayleigh correction for $\lambda$ = {:3.0f} nm".format(wl)
    plt.plot(all_datetime1,-2.5/np.log(10.)*od_adiab,':',color=colors[ii],label=label)
    ii+=1
    
plt.title("Spectro/Photometry magnitude comparison for $\lambda$ = 400 nm (spectro data)")
plt.xlabel("date")
plt.ylabel("Relative magnitude (mag)")
plt.legend(fontsize=15)

In [ ]:
am=t["airmass"]
od_adiab = RayOptDepth_adiabatic(500.,altitude=2890.5, costh=1./am)  -  RayOptDepth_adiabatic(500.,altitude=2890.5, costh=1)# Rayleigh optical depth

# absorption magnitude corrected from Rayleigh attenuation
abs=t["starmag"]-2.5/np.log(10.)*od_adiab

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot(2,1,1)
plt.plot(all_datetime1,t["starmag"]+16.2,'bo',label="star photometry")
plt.plot(all_datetime1,MAttenuation_mean_ALL[all_idx[0],map_photo_to_spectro_idx],'ro',label="spectrometry $\lambda$={:3.0f} nm".format(Lambdas_ref[all_idx[0]]))
plt.ylim(-0.75,0.75)
plt.grid()
plt.legend()
plt.subplot(2,1,2)
plt.plot(all_datetime1,abs+16.2,'bo',label="star photometry")
plt.plot(all_datetime1,MAttenuation_mean_ALL[all_idx[0],map_photo_to_spectro_idx],'ro',label="spectrometry $\lambda$={:3.0f} nm".format(Lambdas_ref[all_idx[0]]))
plt.ylim(-0.75,0.75)
plt.grid()
plt.legend()

In [ ]:
am=t["airmass"]
od_adiab = RayOptDepth_adiabatic(500.,altitude=2890.5, costh=1./am)  # Rayleigh optical depth

# absorption magnitude corrected from Rayleigh attenuation
abs=t["starmag"]-2.5/np.log(10.)*od_adiab

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(all_datetime1,abs+16.35,'bo',label="star photometry")
plt.plot(all_datetime1,MAttenuation_mean_ALL[all_idx[0],map_photo_to_spectro_idx],'ro',label="spectrometry $\lambda$={:3.0f} nm".format(Lambdas_ref[all_idx[0]]))
plt.ylim(-0.75,0.75)
plt.grid()
plt.legend()

# New tables on Magnitude and Magnitude corrected from airmass

In [ ]:
MMag_mean_ALL, MMag_Err_ALL, MAbs_mean_ALL,MAbs_Err_ALL=GetOtherTables(tabledir2)

In [ ]:
MMag_mean_ALL.shape

In [ ]:
idx_ref

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(all_datetime1,MMag_mean_ALL[all_idx[0],map_photo_to_spectro_idx]- MMag_mean_ALL[all_idx[0],idx_ref2]   ,'ro',label="spectrometry Mag $\lambda$={:3.0f} nm".format(Lambdas_ref[all_idx[0]]))
plt.plot(all_datetime1,MAbs_mean_ALL[all_idx[0],map_photo_to_spectro_idx]-MAbs_mean_ALL[all_idx[0],idx_ref2],'bo',label="spectrometry Abs $\lambda$={:3.0f} nm".format(Lambdas_ref[all_idx[0]]))
#plt.ylim(26.5,27.5)
plt.ylim(-0.5,0.5)
plt.grid()
plt.legend()

In [ ]:
Sel_Spectro_MMag_mean_ALL = []
Sel_Spectro_MAbs_mean_ALL = []
Sel_Photo_mean_ALL = []
Sel_dt  = []
Sel_am = []

# loop on Datetime photometry
idx_photo=0
for thedt in all_datetime1:
    # test if the current datetime is selected by Marc
    if thedt in selected_datetime:
        idx_spectro=map_photo_to_spectro_idx[idx_photo]
        #add in the collection
        Sel_Spectro_MMag_mean_ALL.append(MMag_mean_ALL[:,idx_spectro]-MMag_mean_ALL[:,idx_ref2] )
        Sel_Spectro_MAbs_mean_ALL.append(MAbs_mean_ALL[:,idx_spectro]-MAbs_mean_ALL[:,idx_ref2] )
        Sel_Photo_mean_ALL.append(t["starmag"][idx_photo]-t["starmag"][idx_ref])
        Sel_dt.append(thedt)
        Sel_am.append(t["airmass"][idx_photo])
    # increase photometry entry
    idx_photo+=1

In [ ]:
plt.figure(figsize=(20,6))
plt.plot(all_datetime1,MMag_mean_ALL[all_idx[0],map_photo_to_spectro_idx]- MMag_mean_ALL[all_idx[0],idx_ref2],'o',color="grey",label="spectrometry Mag $\lambda$={:3.0f} nm".format(Lambdas_ref[all_idx[0]]))
plt.plot(all_datetime1,MAbs_mean_ALL[all_idx[0],map_photo_to_spectro_idx]-MAbs_mean_ALL[all_idx[0],idx_ref2],'o',color="grey",label="spectrometry Abs $\lambda$={:3.0f} nm".format(Lambdas_ref[all_idx[0]]))
plt.ylim(-0.5,0.5)
plt.plot(Sel_dt,Sel_Photo_mean_ALL,"ro",label="photometry")
#plt.plot(Sel_dt,Sel_Spectro_MAbs_mean_ALL,"ro",label="spectrometry")
plt.grid()
plt.legend()